# Dependency Parsing to Identify the Ingroup and Outgroup
---

In this notebook we are experimenting with dependency parsing to identify the ingroup and outgroup of a text.

For this tasks we use spaCy's dependency matcher to identify the ingroup and outgroup using hypernymy.

Hyponymic relationships are expressed in narrative clauses, therefore, we use the dependency matcher to detect hyponymic narrative clauses from which the ingroup and outgroup can be identified.

We refractor a set of regex defined Hearst Patterns into dependency patterns.

# Initiate the pipeline

%%time
import spacy
from spacy.matcher import DependencyMatcher
from pathlib import Path
from spacy.pipeline import merge_entities

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(merge_entities)

# Test Sentences

texts = [
   "There are works by such authors as Herrick, Goldsmith, and Shakespeare.", # such_NOUN_as
   "There are such benefits as postharvest losses reduction, food increase and soil fertility improvement.", # such_NOUN_as
   "There were bruises, lacerations, or other injuries were not prevalent.",
   "common law countries, including Canada, Australia, and England enjoy toast.", #noun, including noun
   "Many countries, especially France, England and Spain also enjoy toast.", #noun, especially noun
   "The evidence we have gathered all points to a collection of loosely affiliated terrorist organizations known as al Qaeda.",
   "The Aryan himself was probably at first a nomad and became a settler in the course of ages",
   "The Jew has never been a nomad, but always a parasite, battening on the substance of others",
   "We are hunting for terrorist groups, particularly the Taliban and al Qaeda",
   "We are hunting for the IRA, ISIS, al Qaeda and some other terrorist groups, especially the Taliban, Web Scientists and particularly Southampton University",
   "Forty-four percent of patients with uveitis had one or more identifiable signs or symptoms, such as red eye, ocular pain, visual acuity, or photophobia, in order of decreasing frequency.",
   "Other close friends, including Canada, Australia, Germany and France, have pledged forces as the operation unfolds.",
   "The evidence we have gathered all points to a collection of loosely affiliated terrorist organizations known as al Qaeda.",
   "Terrorist groups like al Qaeda depend upon the aid or indifference of governments.",
   "This new law that I sign today will allow surveillance of all communications used by terrorists, including e-mails, the Internet, and cell phones.",
   "From this day forward, any nation that continues to harbor or support terrorism will be regarded by the United States as a hostile regime.",
   "We are looking out for the Taliban, al Qaeda and other terrorist groups",
   "We are looking out for al Qaeda and other terrorist groups, especially the Taliban and the muppets"
]

## Import annotated test dataset

The following import is from the annotated dataset for ingroup and outgroup

import os
import csv
import pandas as pd

filename = "entity_list_gold.txt"
filename = os.path.join(os.getcwd(), filename)

with open(filename, newline = "") as fp:
    data = csv.DictReader(fp, delimiter = '\t') 
    
    display(pd.DataFrame(data))

## Extract relevant rows

The inferred and linked annotations may not yet be computationally feasible without co-reference resolution and entity disambiguation.

Therefore, we extract those entries which are computationally feasible for now.

In a function column, annotations have been made for those sentences which can be computationally analysed.
- Classification means the sentence identifies a ingroup or outgroup
- Elevation or othering means a named entity is being elevated or othered in the text

There are two ways in which as group is identified:
- Noun phrase: "Brother PROPN" - in this case the named entity annotated as a proper noun is given ingroup
- Naming: "terrorist organisation known as PROPN" - in this case the Proper Noun becomes known as a terrorist organisation
- Hypernym: 

import os
import csv
import pandas as pd

filename = "entity_list_gold.txt"
filename = os.path.join(os.getcwd(), filename)

test_data = []

with open(filename, newline = "") as fp:
    data = csv.DictReader(fp, delimiter = '\t') 
    
    for entry in data:
            test_data.append(entry)
            
display(pd.DataFrame(test_data))

In [1]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_md")
from spacy.tokens import Token, Span, Doc

## Create the utility functions

In [744]:
%%time

def doc_dep_graph(doc, strip = False):
    
    ''' 
    Put the graph with entity labels present (see 'tag' and 'label')
    '''
    
    words = [] # the words of a dependency graph
    arcs = [] # the arcs of a dependency graph
    
    for tok in doc:
        
        labels = dict()
        labels["text"] = tok.text
        if tok._.concept:
            labels["tag"] = tok._.concept
        else:
            labels["tag"] = tok.ent_type_
        words.append(labels)
        
#     words = [{"text" : tok.text, "tag": tok.pos_} for tok in doc]
#     print("hasModifier: ", [(tok, tok._.hasModifier) for tok in doc])
#     print("isModifying: ", [(tok, tok._.isModifying) for tok in doc])
    
    # iterate through each token in a doc
    for tok in doc:
        
        # ignore punctuation
        if tok.dep_ in ['punct']:
            continue
    
        if strip and not len(tok._.hasModifier):
            continue

        # if the token head has a method
        # the label becomes the method label
        
        if len(tok._.hasModifier) > 0:
            for mod in tok._.hasModifier:
#                 print(tok, '=>', mod[0], '=>', mod[0][1], '=>', tok.i)
                if mod[0] != "isHyponym":
                    label = mod[0]
                    head = mod[1].i

                    if tok.i < head:
                        arcs.append({
                            "start": tok.i,
                            "end": head,
                            "label": label,
                            "dir": "right"
                        })

                    # if the token index is greater than its head, the token head is the head
                    elif tok.i > head:
                        arcs.append({
                            "start": head,
                            "end": tok.i,
                            "label": label,
                            "dir": "left"
                        })
        else:
            if len(tok._.isModifying):
                label = ""
            else:
                label = tok.dep_
            if tok.i < tok.head.i:
                arcs.append({
                    "start": tok.i,
                    "end": tok.head.i,
                    "label": label,
                    "dir": "left"
                })

            # if the token index is greater than its head, the token head is the head
            elif tok.i > tok.head.i:
                arcs.append({
                    "start": tok.head.i,
                    "end": tok.i,
                    "label": label,
                    "dir": "right"
                })
    return {"words": words, "arcs": arcs}

Wall time: 0 ns


## Setup The Named Concept Recognition

In [787]:
%%time
import spacy

# import pandas as pd

def custom_ents(doc):
    
    """
    returns entities with their modifier as an entity phrase
    """
    ents = []
    
    for ent in doc.ents:
        if ent.root.dep_ in ["amod", "compound"]:
            new_ent = Span(doc, ent.start, ent.root.head.i + 1, label = ent.root.ent_type_)
            ents.append(new_ent)
        else:
            ents.append(ent)
    return ents

class Named_Concept_Recognition:
    
    """
    Pipeline Component for labelling concepts by schema types
    and as an named entity for nouns with a named entity modifier
    """
    
    def __init__(self, nlp):
        
        # snippet of the group schema
        self.group_labels = {
            "Social" : {
                "neutral": {"Person": ["man"],
                            "Community": ["organization", "nomad"],
                           "entity": ["thing"]},
                "ingroup": {"Family": ["brothers"],
                            "Ally": ["friend"]},
                "elevation": {"Positive": ["truthful", "exceptional", "good"]},
                "othering": {"Negative": ["unfair", "hoax"]}},
            "Religious": {"neutral": {"ReligiousGroup" : ["ulema"]}},
            "Commercial": {"neutral": {"Customer": ["passenger"]}},
            "Health" : {
                        "outgroup" : {"Vermin": ["parasite"]}},
            "Security": {"outgroup": {"Criminal": ["terrorist"]}},
            "Political": {"neutral": {"Group": ["authorities", "alliance"]}},
            "Military": {"outgroup": {"Adversary": ["enemy"]}}
        }

        # set the custom labels
        Token.set_extension("concept", default = "", force = True)
        Token.set_extension("attribute", default = "", force = True)
        Token.set_extension("context", default = "", force = True)
        Token.set_extension("modifingTerm", default = "", force = True)
        Token.set_extension("hasModifier", default = [], force = True)
        Token.set_extension("isModifying", default = [], force = True)
        Token.set_extension("get_noun_span", getter = self.get_noun_span, force = True)
    
    def __call__(self, doc):
        
        for token in doc:
            token._.context, token._.attribute, token._.concept = self.get_concept(token)
            token._.modifyingTerm = token
            
        return doc
    
    def get_concept(self, token):
        
        """
        token extension for getting labels from group schema
        """
        for context, attributes in self.group_labels.items():
            for attribute, concepts in attributes.items():
                for concept, terms in concepts.items():
                    if token.lemma_.lower() in terms:
                        return context, attribute, concept
        return None, None, None
    
    def get_noun_span(self, token): # Named Concept Recognition
        
        nominal_deps = ["appos", "acl", "relcl", "det", "predet", "nummod", "amod", "poss", "nmod"]
        adverbial_deps = ["advmod", "advcl", "neg", "npmod"]
        compound_deps = ["compound", "prt", "case", "mark"]
        
        for chunk in token.subtree:
            if chunk.i == token.i:
                return token
            if chunk.dep_ in nominal_deps + adverbial_deps + compound_deps:
                return doc[chunk.i : token.i+1]
            
from spacy.matcher import DependencyMatcher
                
def explain(doc):

    """
    doc extension for retrieving grouping category
    """
    modification = {"elevation": "elevated",
                   "othering": "othered",
                   "outgroup": "othered",
                   "ingroup": "elevated",
                   "neutral": "neutral"}

    for ent in doc.noun_chunks:
        
        # get the ent span to include compound statements
        ent_span = ent.root
        for left in ent_span.lefts:
            if left.dep_ == ["compound", "amod"] or left.ent_type_ or left._.concept:
                ent_span = doc[left.i : ent_span.i + 1]
                break
        
        # get whether the ent has a modifying term
        modifier = ent.root._.modifyingTerm
        
        if ent.root._.concept:
            print(f"'{ent_span}' has a '{modifier._.attribute}' classification where '{modifier}' is an '{modifier._.concept}' phrase from the '{modifier._.context}' context.")
        
        if ent.root._.hasModifier:
        
            for method in ent.root._.hasModifier:
                
                # get the modifier noun span
                noun_span = ''
                noun_span = method[1]._.get_noun_span
                modifier = method[1]._.modifyingTerm

                # if the dependency label is negated
                if method[0] in ["hasNegation"]:
                    print(f"'{ent_span}' is disassociated from '{noun_span}', which is a '{modifier._.attribute}' term from the '{modifier._.context}' context.")
                    continue
                    
                if method[0] in ["hasModifier"] and ent.root.ent_type_:
                    print(f"'{ent_span}' is {modification[method[1]._.attribute]} by the '{spacy.explain(method[1].dep_)} ({method[1].dep_})' term '{method[1]}' from the '{method[1]._.context}' context.'")
                    
                elif method[0] not in ["isHyponym"] and not ent.root._.concept:
                    modifier = method[1]._.modifyingTerm
                    print(f"'{ent_span}' is referred to as '{noun_span}', which is a '{modifier._.attribute}' term from the '{modifier._.context}' context.")
                
                # warning for particular using of terms
                if modifier._.concept == "Vermin":
                    print(f"WARNING: the term '{modifier}' referring to '{ent_span}' is from the '{modifier._.concept}' category and is often used in genocidal language.")
                    
component = "Named_Concept_Recognition"
if component in nlp.pipe_names:
    nlp.remove_pipe(component)
nlp.add_pipe(Named_Concept_Recognition(nlp))
Doc.set_extension("explain", method = explain, force = True)
Doc.set_extension("custom_ents", getter = custom_ents, force = True)

Wall time: 0 ns


## Set Up The Clause Labelling Dependency Parser

In [788]:
%%time
class Clause_Labelling:
    
    def __init__(self, nlp, patterns, pattern_names=None):
         
    # Add patterns with pattern_names to the dependency matcher 
        
        if pattern_names is None:
            pattern_names = ["pattern" + str(pi) for pi in range(len(patterns))]
        else:
            pattern_names = [x for x in pattern_names]
        self.matcher = DependencyMatcher(nlp.vocab)
        for pi, pattern in enumerate(patterns):
            #         print("pattern names: ", pattern_names[pi], pattern)
            self.matcher.add(pattern_names[pi], None, pattern)
        
    def __call__(self, doc): # Clause Labelling
    # Match the patterns to a doc, returns dep graph with edges that match 
    
        matches = []
        # iterate through the matches
        for match in self.matcher(doc):

            # capture the match label
            match_id = nlp.vocab.strings[match[0]]
#             print(f"{match_id}: {[doc[i] for i in match[1][0]]}")
            
            methods = {
                "hasMod": "hasModifier",
                "hasModifier": "isModifierOf",
                
                "isHyponym": "hasHypernym",
                "hasHypernym": "isHyponym",
                "isHyp" : "isHyponym",
                "verbprepPredicate" : "isHyponym",
                "verbPredicate" : "isHyponym",
                
                "hasNameOf": "isCalled",
                "hasNamely": "isCalled",
                "isCalled": "isKnownAs"
            }

            for subtree in match[1]:
                negated = ""
                for i, idx in zip(range(len(subtree)), subtree):
                    pattern = patterns[match_id][i]["SPEC"]["NODE_NAME"]
                    if pattern == "SUBJECT":
                        clause_subject = doc[idx]
                    if pattern == "OBJECT":
                        clause_object = doc[idx]
                    if pattern == "NEGATED":
                        negated = doc[idx]

                if clause_object._.concept:
                    clause_subject._.hasModifier.append((methods[match_id], clause_object))
                    clause_object._.isModifying.append((methods[clause_subject._.hasModifier[-1][0]], clause_subject))
                    
                if match_id in ["hasMod"] and clause_object._.concept:
                    clause_subject._.modifyingTerm = clause_object
                    
                elif match_id not in ["hasMod"]:
                    clause_subject._.isModifying.append((methods[match_id], clause_object))
                    clause_object._.hasModifier.append((methods[clause_subject._.isModifying[-1][0]], clause_subject))
#                     clause_object._.modifyingTerm = clause_subject
                
                if negated:
                    negated._.isModifying.append(("isNegated", clause_object))
                    clause_object._.hasModifier.append(("hasNegation", negated))
#                     clause_object._.modifyingTerm = clause_subject
        return doc

Wall time: 0 ns


## Developing Patterns for the Parser

Developing general patterns for clauses using the labelling schema reveals a primary syntactic model upon whichlayers of meaning are applied according to the predicate relationship. In reference to UD framework categories, one of six subject dependency labels generally refer to a clause’s subject, and one of four object labels refer to the object,while one of two prepositional or one of three complement labels can refer to either. For Labov and Waletzky, predicatesare a clause’s head, which indicates its syntactic function and sequence in a narrative. For detecting a predicate’s syntactic function, the UD framework is also used for verb labelling, while VerbNet provides a lexical resource for interpreting its the semantic meaning. As will be shown, how the subject and object are linked by the predicate gives different meaning to each clause.

In [789]:
%%time

naming_predicate_list = {"LEMMA": {"IN": ["know", "name", "namely", "baptize", "call", "christen",
                         "dub", "entitle", "nickname", "rename"]}}
verb_tag_list = {"TAG": {"IN": ["VBN", "VBD", "VBP", "VBG", "VBZ"]}}
_object = {"POS": {"IN": ["NOUN", "PROPN"]}}
_subject = {"POS": {"IN": ["NOUN", "PROPN", "PRON"]}}
subject_deps = {"DEP": {"IN": ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]}}
object_deps = {"DEP": {"IN": ["dobj", "dative", "attr", "oprd"]}}
complements_deps = {"DEP": {"IN": ["ccomp", "xcomp", "acomp"]}}
nominal_deps = {"DEP": {"IN": ["appos", "acl", "relcl", "det", "predet", "nummod", "amod", "poss", "nmod"]}}
adverbial_deps = {"DEP": {"IN": ["advmod", "advcl", "neg", "npmod"]}}
preposition_deps = {"DEP": {"IN": ["pobj", "pcomp"]}}
coordination_deps = {"DEP": {"IN": ["conj", "cc", "preconj", "prep"]}}
auxiliary_deps = {"DEP": {"IN": ["aux", "auxpass"]}}
compound_deps = {"DEP": {"IN": ["compound", "prt", "case", "mark"]}}
ROOT = {"DEP": {"IN": ["ROOT"]}}
_object = {**_object, **object_deps}
_subject = {**_subject, **subject_deps}

def join_objs(d1, lst):
    DEPS = []
    for entry in lst:
        DEPS += entry["DEP"]["IN"]

    return dict([("POS", d1["POS"]), ("DEP", {"IN": d1["DEP"]["IN"] + DEPS})])

Wall time: 0 ns


## Method 1 - Detects Nouns or Proper Nouns which are modified by a Named Concept

The modification method detects modifier relationships in the dependency parse for both objectives one and two of the methodology. 

To explain this method, we take four wrongly classified named entities from the first Sentiment Analysis experiment
- Muslim friends (Bush (-0.83)), 
- the United States Authorities (Bush (-0.64))
- Mujahidin Brothers (bin Laden (-0.58))
- US Enemy (bin Laden (+0.42)).

The code below shows how using the group labelling schema, each concept is labelled according to its group concept using the custom pipeline component.

In [790]:
texts = [
    "our many Muslim friends. the United States Authorities.",
    "your Mujahidin Brothers. the US Enemy.",
    "the unfair United Nations. the truthful Ulema."
        ]

patterns = {}

patterns.update({"hasMod": [
    
    {"PATTERN": {"POS": {"IN": ["NOUN", "PROPN"]}},
     "SPEC": {"NODE_NAME": "SUBJECT"}},
    
    {"PATTERN": {"DEP": {"IN": ["amod", "compound"]}},
     "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": ">", "NODE_NAME": "OBJECT"}}
    
]})

components = ["Clause_Labelling"]
for component in components:
    if component in nlp.pipe_names:
        nlp.remove_pipe(component)
        
options = {"fine_grained" : True,
          "add_lemma" : True,
          "distance" : 110,
          "word_spacing" : 40,
          "collapse_punct": True}

nlp.add_pipe(Clause_Labelling(nlp, patterns.values(), patterns.keys()))

for text in texts:
    doc = nlp(text)
#     print([(tok, tok._.isModifying) for tok in doc])
    displacy.render(doc_dep_graph(doc, strip = False), style = "dep", manual = True, options = options)
    doc._.explain()

'Muslim friends' has a 'ingroup' classification where 'friends' is an 'Ally' phrase from the 'Social' context.
'the United States Authorities' has a 'neutral' classification where 'Authorities' is an 'Group' phrase from the 'Political' context.


'Mujahidin Brothers' has a 'ingroup' classification where 'Brothers' is an 'Family' phrase from the 'Social' context.
'the US Enemy' has a 'outgroup' classification where 'Enemy' is an 'Adversary' phrase from the 'Military' context.


'unfair United Nations' is othered by the 'adjectival modifier (amod)' term 'unfair' from the 'Social' context.'
'truthful Ulema' has a 'elevation' classification where 'truthful' is an 'Positive' phrase from the 'Social' context.
'truthful Ulema' is elevated by the 'adjectival modifier (amod)' term 'truthful' from the 'Social' context.'


As can be seen from above}, these labels enable the generation of an rationale to explain group classification from a compound or adjectival modifier (amod) relationship. 

As also shown above, this method can also be used to explain how an named entity is either elevated or othered by a modifier term.

## Method 2: Naming Method

While the modification method draws upon a simple modifier relationship, the naming method is the first of the three to draw upon language clauses. 

The pattern for this method is based on the clause head linking a a subject and object by a naming term. 

The predicate terms from the data set for this method are 'known' and 'named', and using VerbNet, 20 similarily functional terms are added to the pattern.

In [791]:
patterns = {}

patterns.update({"hasMod": [
    
    {"PATTERN": {"POS": {"IN": ["NOUN", "PROPN"]}},
     "SPEC": {"NODE_NAME": "SUBJECT"}},
    
    {"PATTERN": {"DEP": {"IN": ["amod", "compound"]}},
     "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": ">", "NODE_NAME": "OBJECT"}}
    
]})

patterns.update({"hasNameOf": [  # object naming object

    # OBJECT known as / named SUBJECT

    {"PATTERN": join_objs(_subject, [preposition_deps]),
     "SPEC": {"NODE_NAME": "SUBJECT"}},

    {"PATTERN": naming_predicate_list,
        "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": ">", "NODE_NAME": "PREDICATE"}},

    {"PATTERN": join_objs(_object, [_subject, preposition_deps]),
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">>", "NODE_NAME": "OBJECT"}}
]})

patterns.update({"hasNamely": [  # object naming object

    # VERB the OBJECT, namely the SUBJECT

    {"PATTERN": _object,
     "SPEC": {"NODE_NAME": "SUBJECT"}},
    
    {"PATTERN": verb_tag_list,
     "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": "<", "NODE_NAME": "VERB_PREDICATE"}},
    
    {"PATTERN": join_objs(_object, [ROOT]),
     "SPEC": {"NBOR_NAME": "VERB_PREDICATE", "NBOR_RELOP": "<", "NODE_NAME": "OBJECT"}},
    
    {"PATTERN": naming_predicate_list,
        "SPEC": {"NBOR_NAME": "OBJECT", "NBOR_RELOP": ">", "NODE_NAME": "PREDICATE"}},
]})

texts = [
    "passengers like an exceptional man named Todd Beamer",
    "a collection of loosely affiliated terrorist organizations known as al Qaeda.",
#     "leaving the main enemy in the region, namely the Jewish-American alliance"
]

components = ["Clause_Labelling"]
for component in components:
    if component in nlp.pipe_names:
        nlp.remove_pipe(component)
        
nlp.add_pipe(Clause_Labelling(nlp, patterns.values(), patterns.keys()))

for text in texts:
    
    doc = nlp(text)
    options = {
#         "fine_grained" : True,
        "add_lemma" : True,
        "distance" : 900/len(doc),
        "word_spacing" : 40,
        "collapse_punct": True}

    displacy.render(doc_dep_graph(doc, strip = True), style = "dep", manual = True, options = options)
    doc._.explain()
#     displacy.render(doc, options = options)

'passengers' has a 'neutral' classification where 'passengers' is an 'Customer' phrase from the 'Commercial' context.
'exceptional man' has a 'elevation' classification where 'exceptional' is an 'Positive' phrase from the 'Social' context.
'Todd Beamer' is referred to as 'an exceptional man', which is a 'elevation' term from the 'Social' context.


'terrorist organizations' has a 'outgroup' classification where 'terrorist' is an 'Criminal' phrase from the 'Security' context.
'al Qaeda' is referred to as 'loosely affiliated terrorist organizations', which is a 'outgroup' term from the 'Security' context.


In [792]:
group_labels = {
    "Social" : {"neutral": {"Person": ["man"],
                            "Community": ["organization", "nomad"],
                            "Entity": ["thing"]},
                "ingroup": {"Family": ["brothers"],
                            "Ally": ["friend"]},
                "elevation": {"Right": ["truthful"],
                              "Good": ["exceptional", "good"]},
                "othering": {"Wrong": ["unfair", "hoax"]}},
    "Religious": {"neutral": {"ReligiousGroup" : ["ulema"]}},
    "Commercial": {"neutral": {"Customer": ["passenger"]}},
    "Health" : {"neutral": {"Entity": ["vaccine"]}, 
                "outgroup" : {"Vermin": ["parasite"]}},
    "Security": {"outgroup": {"Criminal": ["terrorist"]}},
    "Political": {"neutral": {"Group": ["authorities", "alliance"]}},
    "Military": {"outgroup": {"Adversary": ["enemy"]}}
}

def get_concept(token):

    """
    token extension for getting labels from group schema
    """
    for context, attributes in group_labels.items():
        for attribute, concepts in attributes.items():
            for concept, terms in concepts.items():
                if token.lower() in terms:
                    return context, attribute, concept
    return "fuckface"

print(get_concept("terrorist"))

('Security', 'outgroup', 'Criminal')


In [793]:
patterns = {}

patterns.update({"hasMod": [
    
    {"PATTERN": {"POS": {"IN": ["NOUN", "PROPN"]}},
     "SPEC": {"NODE_NAME": "SUBJECT"}},
    
    {"PATTERN": {"DEP": {"IN": ["amod", "compound"]}},
     "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": ">", "NODE_NAME": "OBJECT"}}
    
]})

patterns.update({"hasNameOf": [  # object naming object

    # OBJECT known as / named SUBJECT

    {"PATTERN": join_objs(_subject, [preposition_deps]),
     "SPEC": {"NODE_NAME": "SUBJECT"}},

    {"PATTERN": naming_predicate_list,
        "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": ">", "NODE_NAME": "PREDICATE"}},

    {"PATTERN": join_objs(_object, [_subject, preposition_deps]),
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">>", "NODE_NAME": "OBJECT"}}
]})

patterns.update({"hasNamely": [  # object naming object

    # VERB the OBJECT, namely the SUBJECT

    {"PATTERN": _object,
     "SPEC": {"NODE_NAME": "SUBJECT"}},
    
    {"PATTERN": verb_tag_list,
     "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": "<", "NODE_NAME": "VERB_PREDICATE"}},
    
    {"PATTERN": join_objs(_object, [ROOT]),
     "SPEC": {"NBOR_NAME": "VERB_PREDICATE", "NBOR_RELOP": "<", "NODE_NAME": "OBJECT"}},
    
    {"PATTERN": naming_predicate_list,
        "SPEC": {"NBOR_NAME": "OBJECT", "NBOR_RELOP": ">", "NODE_NAME": "PREDICATE"}},
]})

hyponymy_list = {"LEMMA": {"IN": ["like", "include", "except", "whether", "as"]}}
patterns.update({"isHyp": [  # Hypernym prep Hyponym
    
    {"PATTERN": join_objs(_subject, [ROOT]),
     "SPEC": {"NODE_NAME": "SUBJECT"}},

    {"PATTERN": hyponymy_list,
        "SPEC": {"NBOR_NAME": "SUBJECT", "NBOR_RELOP": ">", "NODE_NAME": "PREDICATE"}},

    {"PATTERN": join_objs(_object, [preposition_deps]),
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">>", "NODE_NAME": "OBJECT"}}
]})

patterns.update({"verbPredicate": [  # Hypernym verb Hyponym

    # who attacked our country
    {"PATTERN": _subject,
     "SPEC": {"NODE_NAME": "OBJECT"}},

    {"PATTERN": verb_tag_list,
     "SPEC": {"NBOR_NAME": "OBJECT", "NBOR_RELOP": "<", "NODE_NAME": "PREDICATE"}},

    {"PATTERN": {"DEP": {"IN": ["neg"]}},
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">", "NODE_NAME": "NEGATION"}},
    
    {"PATTERN": {"DEP": {"IN": ["cc"]}},
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">", "NODE_NAME": "CORDINATING_CONJUNCTION"}},

    {"PATTERN": join_objs(_object, [complements_deps]),
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">", "NODE_NAME": "NEGATED"}},
    
    {"PATTERN": {"DEP": {"IN": ["conj"]}},
     "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">", "NODE_NAME": "SUBJECT"}}
]})

patterns.update({"verbprepPredicate": [  # Hypernym verb (prep) Hyponym

    # war begins with al Qaeda

    {"PATTERN": _subject,
     "SPEC": {"NODE_NAME": "OBJECT"}},

    {"PATTERN": verb_tag_list,
        "SPEC": {"NBOR_NAME": "OBJECT", "NBOR_RELOP": "<", "NODE_NAME": "PREDICATE"}},

    {"PATTERN": {
        "DEP": {"IN": ["prep"]}}, # TODO: this needs to be fixed to be an adjacent token
        "SPEC": {"NBOR_NAME": "PREDICATE", "NBOR_RELOP": ">", "NODE_NAME": "PREP"}},

    {"PATTERN": join_objs(_object, [complements_deps, preposition_deps]),
     "SPEC": {"NBOR_NAME": "PREP", "NBOR_RELOP": ">", "NODE_NAME": "SUBJECT"}}
]})


components = ["Clause_Labelling"]
for component in components:
    if component in nlp.pipe_names:
        nlp.remove_pipe(component)
        
nlp.add_pipe(Clause_Labelling(nlp, patterns.values(), patterns.keys()))
    
texts = [
    "passengers like an exceptional man named Todd Beamer",
    "The Aryan himself was probably at first a Nomad",
    "The Jew has never been a Nomad, but always a parasite",
    "vaccines have never been a good thing, but always a hoax"
]

for text in texts:
    
    doc = nlp(text)
    options = {
#         "fine_grained" : True,
        "add_lemma" : True,
        "distance" : 1000/len(doc),
        "word_spacing" : 40,
        "collapse_punct": True}

    displacy.render(doc_dep_graph(doc, strip = True), style = "dep", manual = True, options = options)
    doc._.explain()
#     displacy.render(doc, options = options)


'passengers' has a 'neutral' classification where 'passengers' is an 'Customer' phrase from the 'Commercial' context.
'exceptional man' has a 'elevation' classification where 'exceptional' is an 'Positive' phrase from the 'Social' context.
'Todd Beamer' is referred to as 'an exceptional man', which is a 'elevation' term from the 'Social' context.
'Todd Beamer' is referred to as 'passengers', which is a 'neutral' term from the 'Commercial' context.


'Aryan' is referred to as 'a Nomad', which is a 'neutral' term from the 'Social' context.
'Nomad' has a 'neutral' classification where 'Nomad' is an 'Community' phrase from the 'Social' context.


'Jew' is referred to as 'a parasite', which is a 'outgroup' term from the 'Health' context.
'Jew' is disassociated from 'a Nomad', which is a 'neutral' term from the 'Social' context.
'Nomad' has a 'neutral' classification where 'Nomad' is an 'Community' phrase from the 'Social' context.
'parasite' has a 'outgroup' classification where 'parasite' is an 'Vermin' phrase from the 'Health' context.


'vaccines' is referred to as 'a hoax', which is a 'othering' term from the 'Social' context.
'vaccines' is disassociated from 'a good thing', which is a 'elevation' term from the 'Social' context.
'good thing' has a 'elevation' classification where 'good' is an 'Positive' phrase from the 'Social' context.
'hoax' has a 'othering' classification where 'hoax' is an 'Negative' phrase from the 'Social' context.
